In [307]:
import pandas as pd
import re
import json

In [308]:
sector_freq_df = pd.read_excel("../data/ukbusinessworkbook2020.xlsx", sheet_name = "Table 1")

In [309]:
sector_freq_df.head(10)

,Unnamed: 0,Unnamed: 1,"Table 1 - Number of VAT and/or PAYE based enterprises in districts, counties and unitary authorities within region and country by broad industry group",Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,NaN,NaN,"UK, 2020",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Broad Industry Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,"01-03 : Agriculture,forestry & fishing",05-39 : Production,41-43 : Construction,45 : Motor trades,46 : Wholesale,47 : Retail,49-53 : Transport & Storage (inc postal),55-56 : Accommodation & food services,58-63 : Information & communication,64-66 : Finance & insurance,68 : Property,"69-75 : Professional, scientific & technical",77-82 : Business administration & support serv...,84 : Public administration & defence,85 : Education,86-88 : Health,"90-99 : Arts, entertainment, recreation & othe...",Total
5,K02000001,United Kingdom,148845,152675,351250,76910,101330,208795,123075,161695,225745,62040,102340,467595,235755,7665,45000,103265,175720,2749700
6,K03000001,Great Britain,130520,147370,340705,74285,98220,203125,120310,157485,223710,60850,99955,461535,232915,7625,44305,100530,171075,2674520
7,K04000001,England and Wales,113185,136600,319755,69640,93060,189740,114390,143050,213185,57535,94080,430690,219655,7570,42285,93945,158460,2496825
8,E92000001,England,99425,129960,306260,65915,89935,181235,109550,134150,208550,55635,91175,418630,211430,7035,40915,89605,151565,2390970
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [310]:
# Create series of region names, offset by 1 forward
# so that when you select the rows that are NaNs
# you in fact select the row AFTER the NaN
region_names = sector_freq_df.iloc[6:450,1]

In [311]:
# Create boolean list indicating location of NaNs
names_index = list(sector_freq_df.iloc[5:449,1].isnull())

In [312]:
region_names_list = list(region_names[names_index])

In [313]:
# Create series of region freqncies, offset by 1 forward
# so that when you select the rows that are NaNs
# you in fact select the row AFTER the NaN
region_frequency = sector_freq_df.iloc[6:450,19]

In [314]:
region_freqency_list = list(region_frequency[names_index])

In [315]:
region_names_list = [x.rstrip() for x in region_names_list]

In [316]:
region_names_list

['North East',
 'North West',
 'Yorkshire and The Humber',
 'East Midlands',
 'West Midlands',
 'East',
 'London',
 'South East',
 'South West',
 'Wales',
 'Scotland',
 'Northern Ireland']

In [317]:
region_freqency_list

[71465,
 267055,
 189265,
 184015,
 221905,
 271395,
 531300,
 418370,
 236200,
 105855,
 177695,
 75180]

In [318]:
# Create region distribution dictionary
region_dist = {} 
for key in region_names_list: 
    for value in region_freqency_list: 
        region_dist[key] = value 
        region_freqency_list.remove(value) 
        break 

## Create joint dist of (region, sector)

In [319]:
# Create list of sector names
sector_name = list(sector_freq_df.iloc[4,2:19])

In [320]:
# Remove unwanted characters from names
sector_name = [" ".join(re.findall("[a-zA-Z]+", x)) for x in sector_name] 

In [321]:
# Create df of region-sector frequencies, offset by 1 forward
# so that when you select the rows that are NaNs
# you in fact select the row AFTER the NaN
region_sector_frequency = sector_freq_df.iloc[6:450,2:19]

In [322]:
# select only the high level regions
region_sector_frequency_high_level = pd.DataFrame(region_sector_frequency[names_index])

In [323]:
# Rename index and columns
region_sector_frequency_high_level.index = region_names_list
region_sector_frequency_high_level.columns = sector_name

In [324]:
region_sector_frequency_high_level

,Agriculture forestry fishing,Production,Construction,Motor trades,Wholesale,Retail,Transport Storage inc postal,Accommodation food services,Information communication,Finance insurance,Property,Professional scientific technical,Business administration support services,Public administration defence,Education,Health,Arts entertainment recreation other services
North East,3950,4920,9525,2395,2115,5835,2890,6430,3345,1190,2270,11245,5540,245,1285,3005,5280
North West,11680,16215,31525,8325,11255,24665,13240,17580,15745,7555,10065,42725,23365,560,4430,11425,16700
Yorkshire and The Humber,11580,13385,23785,6820,7695,15340,12335,13745,10055,3765,6880,26310,14085,645,3125,7825,11890
East Midlands,11030,13135,23905,6710,7525,13665,12800,10830,10015,5095,6460,25450,14155,935,3190,6885,12230
West Midlands,12480,15565,26150,7600,9780,16965,15360,12490,12715,3620,7730,31175,24630,700,3595,8880,12470
East,11935,15665,43505,8710,10550,17995,13940,13270,22185,5235,9510,44580,22840,1350,4755,9395,15975
London,785,16845,57450,6725,19300,40125,15585,24350,72510,15960,25440,120460,51980,100,8910,18530,36245
South East,11785,20485,57980,11155,14470,31050,14910,19780,45685,8560,14250,81095,36995,1250,7685,14865,26370
South West,24200,13745,32435,7475,7245,15595,8490,15675,16295,4655,8570,35590,17840,1250,3940,8795,14405
Wales,13760,6640,13495,3725,3125,8505,4840,8900,4635,1900,2905,12060,8225,535,1370,4340,6895


In [325]:
# Convert region_sector_frequency_high_level df to a dictionary
# with the following structure:
# {"region": {"sector_1": count, "sector_2": count,...}}

region_sector_string = region_sector_frequency_high_level.to_json(orient="index")
region_sector_dist = json.loads(region_sector_string)
#json.dumps(parsed, indent=4) 

## Create joint dist of (region, turnover) 

In [326]:
turnover_freq_df = pd.read_excel("../data/ukbusinessworkbook2020.xlsx", sheet_name = "Table 7")

In [327]:
turnover_freq_df.head(12)

,Unnamed: 0,Table 7 - Number of VAT and/or PAYE based enterprises by parliamentary constituency and turnover sizebands,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,"UK, 2020",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Turnover Size Band (£000's),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0-49,50-99,100-249,250-499,500-999,1000-1999,2000-4999,5000-9999,10000-49999,50000+,Total
5,K02000001: United Kingdom,411835,622655,901405,351580,203020,113960,78985,30090,27290,8880,2749700
6,K03000001: Great Britain,392990,608805,881710,342360,197145,110445,76605,29200,26565,8695,2674520
7,K04000001: England and Wales,367095,569300,822570,318560,183715,103065,71900,27470,24970,8180,2496825
8,E92000001 : England,347275,545610,789750,304825,176180,99140,69475,26580,24180,7955,2390970
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [328]:
# Create list of turnover bands
turnover_bands = list(turnover_freq_df.iloc[4,1:11])

In [329]:
# Replace NaNs with ""
turnover_freq_df["Unnamed: 0"] = turnover_freq_df["Unnamed: 0"].fillna("")

In [330]:
# Remove unwanted characters 
turnover_freq_df["Unnamed: 0"] = [" ".join(re.findall("[a-zA-Z]+", x))[2:] for x in turnover_freq_df.iloc[:,0]]

In [331]:
# Create df of region-turnover frequencies
region_turnover_frequency = turnover_freq_df.iloc[5:683,1:11]

In [332]:
# Select only those rows for the high level regions
region_turnover_frequency_high_level = region_turnover_frequency[list(turnover_freq_df.iloc[5:683,0].isin(region_names_list))]

In [333]:
region_turnover_frequency_high_level

,Table 7 - Number of VAT and/or PAYE based enterprises by parliamentary constituency and turnover sizebands,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
10,8665,17100,24380,9625,5250,2870,1925,715,705,225
41,38065,64620,85755,33720,19475,11180,7770,3015,2635,820
118,26750,43855,61000,25090,14115,8180,5570,2165,2005,535
174,29320,42970,57585,23330,13540,7530,5530,2020,1695,495
222,31995,50790,68580,30095,18985,9695,6500,2445,2170,655
283,37720,62925,90200,34580,20055,11380,8025,3075,2680,755
343,74775,118030,181540,65000,37810,22225,16130,6635,6545,2605
418,60645,93400,144580,51765,29655,16760,11820,4365,4010,1375
504,39340,51920,76130,31620,17295,9320,6205,2145,1735,490
561,19820,23690,32820,13735,7535,3925,2425,890,790,225


In [334]:
region_names_list

['North East',
 'North West',
 'Yorkshire and The Humber',
 'East Midlands',
 'West Midlands',
 'East',
 'London',
 'South East',
 'South West',
 'Wales',
 'Scotland',
 'Northern Ireland']

In [335]:
# Rename index and columns
region_turnover_frequency_high_level.index = region_names_list
region_turnover_frequency_high_level.columns = turnover_bands

In [336]:
region_turnover_frequency_high_level

,0-49,50-99,100-249,250-499,500-999,1000-1999,2000-4999,5000-9999,10000-49999,50000+
North East,8665,17100,24380,9625,5250,2870,1925,715,705,225
North West,38065,64620,85755,33720,19475,11180,7770,3015,2635,820
Yorkshire and The Humber,26750,43855,61000,25090,14115,8180,5570,2165,2005,535
East Midlands,29320,42970,57585,23330,13540,7530,5530,2020,1695,495
West Midlands,31995,50790,68580,30095,18985,9695,6500,2445,2170,655
East,37720,62925,90200,34580,20055,11380,8025,3075,2680,755
London,74775,118030,181540,65000,37810,22225,16130,6635,6545,2605
South East,60645,93400,144580,51765,29655,16760,11820,4365,4010,1375
South West,39340,51920,76130,31620,17295,9320,6205,2145,1735,490
Wales,19820,23690,32820,13735,7535,3925,2425,890,790,225


In [337]:
# Convert region_turnover_frequency_high_level df to a dictionary
# with the following structure:
# {"region": {"turnover_1": count, "turnover_2": count,...}}
region_turnover_string = region_turnover_frequency_high_level.to_json(orient="index")
region_turnover_dist = json.loads(region_turnover_string)

## Create entity dictionary and save it

In [338]:
# Create dictionary with all entity parameters
entity_dict = {"business":{
    "region_dist" : region_dist,
    "region_sector_dist" : region_sector_dist,
    "region_turnover_dist": region_turnover_dist
}}

In [339]:
# Write entity parameter dict to json
with open("entity_params.json", "w") as outfile:  
    json.dump(entity_dict, outfile, indent = 4) 

In [1]:
import csv

In [8]:
file = "/Users/samhastings/Documents/coding/sandbox/data/transactions/run03/4.csv"
fileObject = csv.reader(file)
row_count = sum(1 for row in fileObject)  # fileObject is your csv.reader

In [9]:
row_count

73